# KERAS

In [1]:
# 패키지
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Dropout


Using TensorFlow backend.


In [2]:
from keras import backend as K
print(K.image_data_format())
# channels_last: [N, H, W, C]
# channels_first: [N, C, H, W]

channels_last


In [3]:
# parameters
batch_size = 100
num_classes = 10
training_epochs = 10

# input image dimension
img_rows, img_cols = 28, 28

# MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)


if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train /= 255
x_test /= 255
print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)

print("y_train[0:10]=", y_train[0:10])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train.shape: ", y_train.shape)
print("y_train[0:10]=", y_train[0:10])

x_train.shape:  (60000, 28, 28)
y_train.shape:  (60000,)
x_test.shape:  (10000, 28, 28)
y_test.shape:  (10000,)
x_train.shape:  (60000, 28, 28, 1)
y_train.shape:  (60000,)
y_train[0:10]= [5 0 4 1 9 2 1 3 1 4]
y_train.shape:  (60000, 10)
y_train[0:10]= [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [4]:
%%time
# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Loss & Optimize
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Train
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=training_epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Test
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 116s 2ms/step - loss: 2.3016 - acc: 0.1108 - val_loss: 2.3012 - val_acc: 0.1135
Epoch 2/10
60000/60000 [==============================] - 114s 2ms/step - loss: 2.3014 - acc: 0.1118 - val_loss: 2.3011 - val_acc: 0.1135
Epoch 3/10
60000/60000 [==============================] - 117s 2ms/step - loss: 2.0749 - acc: 0.2230 - val_loss: 0.6240 - val_acc: 0.8048
Epoch 4/10
60000/60000 [==============================] - 118s 2ms/step - loss: 0.3809 - acc: 0.8810 - val_loss: 0.1780 - val_acc: 0.9445
Epoch 5/10
60000/60000 [==============================] - 121s 2ms/step - loss: 0.1771 - acc: 0.9456 - val_loss: 0.1102 - val_acc: 0.9674
Epoch 6/10
60000/60000 [==============================] - 111s 2ms/step - loss: 0.1217 - acc: 0.9623 - val_loss: 0.0859 - val_acc: 0.9710
Epoch 7/10
60000/60000 [==============================] - 122s 2ms/step - loss: 0.0967 - acc: 0.9702 - val_loss: 0.0689 - 

In [5]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
__________

In [6]:
# GraphViz install (https://www.graphviz.org)
# pip install pydot
keras.utils.vis_utils.plot_model(model, 
                                 to_file='cnn.png',
                                show_shapes=True,
                                show_layer_names=True)

<img src='cnn.png'>